In [1]:
!git clone https://github.com/taehojo/data.git

Cloning into 'data'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 3), reused 20 (delta 2), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [7]:
import pandas as pd
df = pd.read_csv('/content/data/sonar3.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0


In [8]:
df[60].value_counts()

1    111
0     97
Name: 60, dtype: int64

In [18]:
X = df.iloc[:,0:60]
y = df.iloc[:,60]

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [20]:
model = Sequential()
model.add(Dense(24, input_dim=60, activation = 'relu'))
model.add(Dense(10, activation ='relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [21]:
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
history = model.fit(X, y, epochs = 200, batch_size=10)

Epoch 1/200
21/21 [==============================] - 1s 2ms/step - loss: 0.7062 - accuracy: 0.4808
Epoch 2/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6838 - accuracy: 0.5721
Epoch 3/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6684 - accuracy: 0.6442
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6568 - accuracy: 0.6490
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6457 - accuracy: 0.6635
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6322 - accuracy: 0.6827
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6153 - accuracy: 0.6683
Epoch 8/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6025 - accuracy: 0.6923
Epoch 9/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5815 - accuracy: 0.7019
Epoch 10/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5687 - accuracy: 0.7212
Epoch 11/

## 학습셋과 테스트셋 구분

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
X = df.iloc[:, 0:60] 
y = df.iloc[:, 60]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

model = Sequential()
model.add(Dense(24, input_dim=60, activation ='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=200, batch_size=10)

score= model.evaluate(X_test, y_test)
print('Test accuracy: ', score[1])

## 모델 저장과 재사용

In [26]:
model.save('save_model.hdf5')

In [29]:
del model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=200, batch_size=10)

score= model.evaluate(X_test, y_test)
print('Test accuracy: ', score[1])

In [32]:
from tensorflow.keras.models import Sequential, load_model
model = load_model('/content/save_model.hdf5')

## k겹 교차 검증

In [37]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
k = 5

X = df.iloc[:, 0:60]
y = df.iloc[:, 60]

kfold = KFold(n_splits=k, shuffle=True)
acc_score = []

def model_fn():
  model = Sequential()
  model.add(Dense(24, input_dim=60, activation='relu'))
  model.add(Dense(10, activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))
  return model

# 사이킷 런에서는 그리드서치cv로 사용했으나 여기서는 수기로 작업
for train_index, test_index in kfold.split(x):
  X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  model = model_fn()
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  # verbose => 출력 생략, 결과값만 보여줌 
  history = model.fit(X_train, y_train, epochs=200, batch_size = 10, verbose =0)

  accuracy = model.evaluate(X_test, y_test)[1]
  acc_score.append(accuracy)

avg_acc_score = sum(acc_score) / k 

print('정확도: ', acc_score)
print('정확도 평균:', avg_acc_score)

2/2 [==============================] - 0s 7ms/step - loss: 1.3551 - accuracy: 0.7561
정확도:  [0.8809523582458496, 0.761904776096344, 0.761904776096344, 0.8048780560493469, 0.7560975551605225]
정확도 평균: 0.7931475043296814
